This script does lump of work in cleaning and transforming the data. The data before fiscal year 2074/75 is discarded due to non-uniformity. The monthly data has values cumulated to that month from start of fiscal year. So data had to be transformed accordingly, only then it could be merged. So, this script handles tasks including:

- Reading excel files for each fiscal year, month wise
- Cleaining and structuring the data into a structured format: `hscode | description | unit | quantity | value | revenue`
- Subtracting values from previous month to get actual values for the month.
- Merging the data of all months in fiscal year.
- Tagging fiscal year and month in data.
- Merging the data for every fiscal year to produce unified csv of format: `fy | month | hscode | description | unit | quantity | value | revenue`

In [1]:
import os
import pandas as pd
import re
from difflib import get_close_matches

In [2]:
# slugified month names are one of these, so a manually mapped dict is used to get month number
months = {
    'vaishaakh':1,
    'jetth':2,
    'jessth':2,
    'asaadh':3,
    'shraavnn':4,
    'bhaadr':5,
    'aashvin':6,
    'asoj':6,
    'kaartik':7,
    'mangshir':8,
    'maarg': 8,
    'pauss':9,
    'maagh':10,
    'phaagun':11,
    'caitr':12
}

In [3]:
# each dir inside data dir is treated as dir containing data for a fiscal year
fy_dirs = os.listdir('data')
# omit years preceeding 2074 due to un-uniformity of data
fy_dirs = [fy for fy in fy_dirs if int(fy.split('-')[1]) >2073]
fy_dirs.sort()
fy_dirs

['fy-2074-75',
 'fy-2075-76',
 'fy-2076-77',
 'fy-2077-78',
 'fy-2078-79',
 'fy-2079-80',
 'fy-2080-81']

In [4]:
def prepare_file_details(fy_dir):
    """
    prepare_file_details parses files in fy_dir, and creates structured file detail for every file.
    The file detail contains path to file, data year, month and fiscal year

    A sorted list of file details is returned. 
    """
    base_dir = os.path.join('data', fy_dir)
    files = os.listdir(base_dir)
    file_details =[] 
    for file in files:
        matches = re.match(r'(\d{4})-(\w+).xlsx', file)
        if matches:
            year, month = matches.group(1), matches.group(2)
            month_match = get_close_matches(month, months.keys())
            if month_match:
                month = months[month_match[0]]
            else:
                raise Exception("unable to parse month from title", month)
            file_details.append({
                'path': os.path.join(base_dir, file),
                'year': year,
                'month': month,
                'fy': fy_dir.replace('-', '/')[3:]
            })

    custom_month_order = [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
    file_details = sorted(file_details, key=lambda x: (x['year'], custom_month_order.index(x['month'])))
    return file_details

In [5]:
def get_df(path:str):
    """
    get_df returns dataframe by reading file and adjusting header in correct index.
    """
    df = pd.DataFrame({'Unnamed':[1,2,3]})
    i = 0
    while df.columns.str.contains('^Unnamed').sum() > 0:
        df = pd.read_excel(path, sheet_name=5, header=i)
        if len(df.columns) == 5:
            df = pd.read_excel(path,sheet_name=4,header=i)
        i+=1
    return df

In [6]:
def filter_rows(df:pd.DataFrame):
    """filter_rows filters rows in df and only keeps those with hscode starting with 87. i.e. Transport vehicle related data is filtered"""
    filtered = df[df['hscode'].apply(lambda x: str(x).startswith('87'))]
    return filtered

In [7]:
def merge_monthwise_files(file_details):
    """
    merge_monthwise_files does following jobs:

    i. Reads df from file for every month in fiscal year.

    ii. Assigns simple column names.

    iii. Subtracts cumulated values from current month with cumulated values from previous month to get exact figures.

    iv. Filters rows to keep only transport vehicle related rows.

    v. Tag month and fiscal year to monthly data.

    vi. Merge all months dfs into yearly dataframe and returns it.

    """
    dfs = []
    cumulated_cols = ['quantity', 'value', 'revenue']
    for i, file_info in enumerate(file_details):
        print('Processing file:', file_info['path'])
        df = get_df(file_info['path'])
        df.columns = ['hscode', 'description', 'unit', *cumulated_cols]

        if i > 0:
            prev_df = get_df(file_details[i-1]['path'])
            prev_df.columns = ['hscode', 'description', 'unit', *cumulated_cols]
            df_temp = df.set_index('hscode')
            prev_df_temp = prev_df.set_index('hscode')

            # Drop duplicate values from the index
            df_temp = df_temp[~df_temp.index.duplicated(keep='first')]
            prev_df_temp = prev_df_temp[~prev_df_temp.index.duplicated(keep='first')]

            df_temp[cumulated_cols] = df_temp[cumulated_cols].subtract(prev_df_temp[cumulated_cols], fill_value=0)
            df_temp.reset_index(inplace=True)
            
            df = df_temp

        df = filter_rows(df)

        df = df.copy()

        # df['year'] = file_info['year']
        df['month'] = file_info['month']
        df['fy'] = file_info['fy']
        dfs.append(df)
    
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.dropna(inplace=True)
    merged_df = merged_df[['fy', 'month', 'hscode', 'description', 'unit', 'quantity', 'value', 'revenue']]
    # merged_df.to_csv(file_details[0]['path'].split('/')[1]+'.csv', index=False)
    return merged_df

In [8]:
def remove_negatives(df:pd.DataFrame):
    """
    remove_negatives removes -ve values from dataset.

    Due to faults in DoC dataset, some rows have -ve values. Those values are subsided gracefully.

    If values are negligibly -ve ie. -0.0001 type. Then those values are round up to 0.

    Rows with largely negative values are dropped.
    """
    df['quantity'] = df['quantity'].apply(lambda x: x if x > -1 else pd.NA)
    df['value'] = df['value'].apply(lambda x: x if x > -1 else pd.NA)
    df['revenue'] = df['revenue'].apply(lambda x: x if x > -1 else pd.NA)

    df.dropna(inplace=True)

    df['value'] = df['value'].apply(lambda x: max(x, 0))
    df['quantity'] = df['quantity'].apply(lambda x: max(x, 0))
    df['revenue'] = df['revenue'].apply(lambda x: max(x, 0))

    return df

For each fiscal year:

- First prepare file details for the fiscal year directory
- Then merge the monthly data to prepare yearly dataframe
- Merge all yearly dataframes to create final dataframe

In [9]:
yearly_dfs = []

for fy_dir in fy_dirs:
    file_details = prepare_file_details(fy_dir)
    yearly_df = merge_monthwise_files(file_details)    
    yearly_dfs.append(yearly_df)

final_df = pd.concat(yearly_dfs, ignore_index=True)

# some hscodes are interpreted as floats
final_df['hscode'] = final_df['hscode'].apply(lambda x: int(x))

# output csv
final_df.to_csv('extracted.csv', index=False)

Processing file: data/fy-2074-75/2074-shraavnnsmm.xlsx
Processing file: data/fy-2074-75/2074-bhaadrsmm.xlsx
Processing file: data/fy-2074-75/2074-aashvinsmm.xlsx
Processing file: data/fy-2074-75/2074-kaartiksmm.xlsx
Processing file: data/fy-2074-75/2074-maargsmm.xlsx
Processing file: data/fy-2074-75/2074-pausssmm.xlsx
Processing file: data/fy-2074-75/2074-maaghsmm.xlsx
Processing file: data/fy-2074-75/2074-phaagunsmm.xlsx
Processing file: data/fy-2074-75/2074-caitrsmm.xlsx
Processing file: data/fy-2074-75/2075-vaishaakhsmm.xlsx
Processing file: data/fy-2074-75/2075-jesstthsmm.xlsx
Processing file: data/fy-2075-76/2075-shraavnnsmm.xlsx
Processing file: data/fy-2075-76/2075-bhaadrsmm.xlsx
Processing file: data/fy-2075-76/2075-aashvinsmm.xlsx
Processing file: data/fy-2075-76/2075-kaartiksmm.xlsx
Processing file: data/fy-2075-76/2075-maargsmm.xlsx
Processing file: data/fy-2075-76/2075-pausssmm.xlsx
Processing file: data/fy-2075-76/2075-maaghsmm.xlsx
Processing file: data/fy-2075-76/2075-ph

This applies the `remove_negatives` function to handle negative values

In [10]:
negative_df = pd.read_csv('extracted.csv')
positive_df = remove_negatives(negative_df)
positive_df.to_csv('extracted.csv', index=False)